In [ ]:
import paramiko

ip = "172.16.12.11"
user = "root"
pw = "asd123!@"

def cc(cmd):

    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    return out + err


def mariaDB_install():

    cmd =  """
    sudo dnf install -y expect
    dnf -y install dnf-plugins-core epel-release expect
    dnf install -y https://rpms.remirepo.net/enterprise/remi-release-9.rpm
    dnf module reset php -y
    dnf module enable php:remi-8.4 -y
    """
    print(cc(cmd))

def apache_php_install():

    #  Apache + PHP 설치

    cmd1 = r"""
    bash -c 'dnf -y install httpd php php-fpm php-mysqlnd php-cli php-common php-devel \
        php-mbstring php-xml php-gd php-json php-intl php-zip php-opcache'
    """
    cc(cmd1)
    print(cc("systemctl enable --now httpd php-fpm"))
    print(cc("systemctl status httpd php-fpm"))

def install_mariaDB() :
    cmd = """
    sudo dnf install -y mariadb-server
    sudo systemctl enable mariadb --now
    sudo systemctl status mariadb
    sudo dnf install -y mariadb-server-utils
    """
    cc(cmd)
def init_mariaDB() :
    print("secure_installation 시작")
    MYSQL_ROOT_PASSWORD="asd123!@"
    cmd = f'''
cat <<EOF > /root/mysql_secure.exp
#!/usr/bin/expect -f
set timeout -1
set password [lindex $argv 0]
spawn mysql_secure_installation
expect "Enter current password for root (enter for none):"
send "\\r"
expect "Switch to unix_socket authentication"
send "Y\\r"
expect "Change the root password?"
send "y\\r"
expect "New password:"
send "{MYSQL_ROOT_PASSWORD}\\r"
expect "Re-enter new password:"
send "{MYSQL_ROOT_PASSWORD}\\r"
expect "Remove anonymous users?"
send "Y\\r"
expect "Disallow root login remotely?"
send "Y\\r"
expect "Remove test database and access to it?"
send "Y\\r"
expect "Reload privilege tables now?"
send "Y\\r"
expect eof
EOF

chmod +x /root/mysql_secure.exp
/usr/bin/expect /root/mysql_secure.exp "{MYSQL_ROOT_PASSWORD}"

'''
    print(cc(cmd))

def maria_in() :
   mariaDB_install()
   apache_php_install()
   install_mariaDB()
   init_mariaDB()
maria_in()

Last metadata expiration check: 0:02:13 ago on Mon Nov 17 14:44:26 2025.
Package expect-5.45.4-16.el9.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!
Last metadata expiration check: 0:02:14 ago on Mon Nov 17 14:44:26 2025.
Package dnf-plugins-core-4.3.0-20.el9.noarch is already installed.
Package epel-release-9-10.el9.noarch is already installed.
Package expect-5.45.4-16.el9.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!
Last metadata expiration check: 0:02:14 ago on Mon Nov 17 14:44:26 2025.
remi-release-9.rpm                               29 kB/s |  33 kB     00:01    
Package remi-release-9.6-1.el9.remi.noarch is already installed.
Dependencies resolved.
Nothing to do.
Complete!
Last metadata expiration check: 0:02:16 ago on Mon Nov 17 14:44:26 2025.
Dependencies resolved.
Nothing to do.
Complete!
Last metadata expiration check: 0:02:17 ago on Mon Nov 17 14:44:26 2025.
Dependencies resolved.
 Package           Architecture  